In [31]:
import cv2
import numpy as np
#创建一个大小800*800的空帧
frame = np.zeros((800,800,3),np.uint8)
#初始化测量坐标和鼠标运动预测的数组
last_measurement = current_measurement = np.array((0,0),np.float32)
last_predicition = current_prediction = np.array((0,0,0,0),np.float32).reshape(4,1)
#print(last_measurement)
'''
    mousemove()函数在这里的作用就是传递X,Y的坐标值，便于对轨迹进行卡尔曼滤波
'''
def mousemove(event,x,y,s,p):
    #定义全局变量
    global frame,current_measurement,measurements,last_measurement,current_prediction,last_prediction
    #初始化
    last_measurement = current_measurement
    last_prediction = current_prediction
    print(last_prediction)
    #传递当前测量坐标值
    current_measurement = np.array([[np.float32(x)],[np.float32(y)]])
    #用来修正卡尔曼滤波的预测结果
    kalman.correct(current_measurement)
    # 调用kalman这个类的predict方法得到状态的预测值矩阵，用来估算目标位置
    current_prediction = kalman.predict()
    #上一次测量值
    lmx,lmy = last_measurement[0],last_measurement[1]
    #当前测量值
    cmx,cmy = current_measurement[0],current_measurement[1]
    #上一次预测值
    lpx,lpy = last_prediction[0],last_prediction[1]
    #当前预测值
    cpx,cpy = current_prediction[0],current_prediction[1]
    #绘制测量值轨迹（绿色）
    #print(lpx,lpy,cpx,cpy)
    cv2.line(frame,(lmx,lmy),(cmx,cmy),(0,100,0))
    #绘制预测值轨迹（红色）
    cv2.line(frame,(lpx,lpy),(cpx,cpy),(0,0,200))
 
cv2.namedWindow("kalman_tracker")
#调用函数处理鼠标事件，具体事件必须由回调函数的第一个参数来处理，该参数确定触发事件的类型（点击和移动）
'''
void setMousecallback(const string& winname, MouseCallback onMouse, void* userdata=0)
       winname:窗口的名字
       onMouse:鼠标响应函数，回调函数。指定窗口里每次鼠标时间发生的时候，被调用的函数指针。
                这个函数的原型应该为void on_Mouse(int event, int x, int y, int flags, void* param);
       userdate：传给回调函数的参数
 void on_Mouse(int event, int x, int y, int flags, void* param);
        event是 CV_EVENT_*变量之一
        x和y是鼠标指针在图像坐标系的坐标（不是窗口坐标系）
        flags是CV_EVENT_FLAG的组合， param是用户定义的传递到setMouseCallback函数调用的参数。
    常用的event：
        CV_EVENT_MOUSEMOVE
        CV_EVENT_LBUTTONDOWN
        CV_EVENT_RBUTTONDOWN
        CV_EVENT_LBUTTONUP
        CV_EVENT_RBUTTONUP
        和标志位flags有关的：
        CV_EVENT_FLAG_LBUTTON
'''
cv2.setMouseCallback("kalman_tracker",mousemove)
'''
Kalman这个类需要初始化下面变量：
转移矩阵，测量矩阵，控制向量(没有的话，就是0)，
过程噪声协方差矩阵，测量噪声协方差矩阵，
后验错误协方差矩阵，前一状态校正后的值，当前观察值。
    在此cv2.KalmanFilter(4,2)表示转移矩阵维度为4，测量矩阵维度为2
卡尔曼滤波模型假设k时刻的真实状态是从(k − 1)时刻的状态演化而来，符合下式：
            X(k) = F(k) * X(k-1) + B(k)*U(k) + W（k）
其中
F(k)  是作用在xk−1上的状态变换模型（/矩阵/矢量）。 
B(k)  是作用在控制器向量uk上的输入－控制模型。 
W(k)  是过程噪声，并假定其符合均值为零，协方差矩阵为Qk的多元正态分布。
'''
kalman = cv2.KalmanFilter(4,2)
#设置测量矩阵
kalman.measurementMatrix = np.array([[1,0,0,0],[0,1,0,0]],np.float32)
#设置转移矩阵
kalman.transitionMatrix = np.array([[1,0,1,0],[0,1,0,1],[0,0,1,0],[0,0,0,1]],np.float32)
#设置过程噪声协方差矩阵
kalman.processNoiseCov = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]],np.float32)*0.03
 
while True:
    cv2.imshow("kalman_tracker",frame)
    if (cv2.waitKey(30) & 0xff) == 27:
        break
 
cv2.destroyAllWindows()

[[0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]]
[[23.184465]
 [ 6.029126]
 [ 0.      ]
 [ 0.      ]]
[[107.49456 ]
 [ 27.901497]
 [ 21.232126]
 [  5.508201]]
[[303.62033 ]
 [ 78.337875]
 [ 69.71617 ]
 [ 17.9632  ]]
[[573.6709  ]
 [148.0599  ]
 [121.69306 ]
 [ 31.392065]]
[[805.9353  ]
 [207.7953  ]
 [148.19795 ]
 [ 38.186203]]
[[942.9205 ]
 [242.15944]
 [145.664  ]
 [ 37.32246]]
[[996.8163  ]
 [255.7145  ]
 [125.61224 ]
 [ 32.129177]]
[[997.0531  ]
 [254.89618 ]
 [ 98.59833 ]
 [ 25.030172]]
[[968.76855 ]
 [247.40094 ]
 [ 71.33464 ]
 [ 18.041344]]
[[928.8973  ]
 [236.64468 ]
 [ 47.39706 ]
 [ 11.842528]]
[[887.54565  ]
 [225.3241   ]
 [ 28.245178 ]
 [  6.8439507]]
[[849.8913   ]
 [214.99774  ]
 [ 13.997882 ]
 [  3.1317763]]
[[8.1937903e+02]
 [2.0698737e+02]
 [4.3654737e+00]
 [7.2051162e-01]]
[[ 7.9651483e+02]
 [ 2.0125706e+02]
 [-1.5295084e+00]
 [-6.7603177e-01]]
[[780.58405  ]
 [196.90532  ]
 [ -4.6474524]
 [ -1.471843 ]]
[[770.30493  ]
 [194.31389  ]
 [ -5.8666873]
 [ -1.7142284]]
[[

[[357.37946  ]
 [251.818    ]
 [  2.073979 ]
 [  1.0223082]]
[[358.05502  ]
 [251.18416  ]
 [  1.7712595]
 [  0.6637972]]
[[3.5685541e+02]
 [2.4997659e+02]
 [1.1281488e+00]
 [2.5870186e-01]]
[[ 3.5454230e+02]
 [ 2.4848593e+02]
 [ 3.8321149e-01]
 [-1.1998657e-01]]
[[ 3.5166824e+02]
 [ 2.4749265e+02]
 [-3.2189125e-01]
 [-3.0902991e-01]]
[[348.60278   ]
 [246.30637   ]
 [ -0.915795  ]
 [ -0.49892738]]
[[344.9819   ]
 [245.03967  ]
 [ -1.5013708]
 [ -0.665126 ]]
[[339.96494  ]
 [243.76352  ]
 [ -2.262401 ]
 [ -0.7973954]]
[[335.3723    ]
 [243.10512   ]
 [ -2.7668285 ]
 [ -0.76730984]]
[[330.62357   ]
 [242.86374   ]
 [ -3.1958616 ]
 [ -0.65346104]]
[[325.2981   ]
 [242.87805  ]
 [ -3.6568592]
 [ -0.5089035]]
[[ 3.1911520e+02]
 [ 2.4361624e+02]
 [-4.2036719e+00]
 [-2.3894478e-01]]
[[ 3.1366840e+02]
 [ 2.4477824e+02]
 [-4.4727726e+00]
 [ 6.4321995e-02]]
[[308.8028    ]
 [246.14832   ]
 [ -4.5578074 ]
 [  0.34697846]]
[[303.77316   ]
 [248.17125   ]
 [ -4.6599407 ]
 [  0.70977575]]
[[299.246

 [-5.96623182e+00]]
[[ 1.1145085e+02]
 [ 3.4574408e+02]
 [ 1.3943434e-01]
 [-5.8768373e+00]]
[[ 1.12500725e+02]
 [ 3.40017670e+02]
 [ 3.36520284e-01]
 [-5.84427834e+00]]
[[113.718376  ]
 [334.75073   ]
 [  0.52726126]
 [ -5.719304  ]]
[[114.998856  ]
 [329.76562   ]
 [  0.69031215]
 [ -5.5603695 ]]
[[116.86525   ]
 [325.51843   ]
 [  0.94490194]
 [ -5.276108  ]]
[[119.06476  ]
 [320.52533  ]
 [  1.2164894]
 [ -5.214842 ]]
[[122.00631  ]
 [315.58945  ]
 [  1.5899171]
 [ -5.1544533]]
[[125.35564  ]
 [311.26398  ]
 [  1.9707807]
 [ -4.975    ]]
[[128.29282  ]
 [306.72156  ]
 [  2.17998  ]
 [ -4.8813615]]
[[130.88841  ]
 [303.17926  ]
 [  2.2699494]
 [ -4.591493 ]]
[[133.81165  ]
 [299.6578   ]
 [  2.411368 ]
 [ -4.3598547]]
[[136.33371  ]
 [297.26218  ]
 [  2.4353306]
 [ -3.9346538]]
[[139.74834  ]
 [294.93655  ]
 [  2.6473196]
 [ -3.586342 ]]
[[142.54356  ]
 [293.7383   ]
 [  2.6793368]
 [ -3.069382 ]]
[[145.49115  ]
 [292.58585  ]
 [  2.7374055]
 [ -2.654423 ]]
[[148.5276   ]
 [291.3502

[[433.4026   ]
 [354.8379   ]
 [ -6.657873 ]
 [ -0.7612716]]
[[ 4.2650812e+02]
 [ 3.5593503e+02]
 [-6.7090912e+00]
 [-3.5898137e-01]]
[[ 4.1950040e+02]
 [ 3.5737735e+02]
 [-6.7737350e+00]
 [ 3.0950811e-02]]
[[ 4.1243256e+02]
 [ 3.5894968e+02]
 [-6.8373966e+00]
 [ 3.6460990e-01]]
[[405.92865  ]
 [360.5193   ]
 [ -6.7652054]
 [  0.6254564]]
[[399.7931   ]
 [362.6027   ]
 [ -6.628906 ]
 [  0.9410579]]
[[393.87347  ]
 [365.54037  ]
 [ -6.4753604]
 [  1.3732706]]
[[388.64786  ]
 [368.9469   ]
 [ -6.204819 ]
 [  1.8134115]]
[[383.8254   ]
 [372.55466  ]
 [ -5.905575 ]
 [  2.2018332]]
[[379.78555  ]
 [376.78134  ]
 [ -5.5016966]
 [  2.640157 ]]
[[376.7607   ]
 [380.72543  ]
 [ -4.9655256]
 [  2.9224193]]
[[374.28662  ]
 [385.57233  ]
 [ -4.4261947]
 [  3.3390167]]
[[372.63052  ]
 [390.3381   ]
 [ -3.8265486]
 [  3.6478703]]
[[371.37195  ]
 [395.55038  ]
 [ -3.2706547]
 [  3.9865222]]
[[370.82123  ]
 [400.97656  ]
 [ -2.6818643]
 [  4.298167 ]]
[[370.59525  ]
 [407.05164  ]
 [ -2.150232 ]
 [  

 [  3.9489481]]
[[433.0803   ]
 [603.36346  ]
 [  4.5139337]
 [  3.821035 ]]
[[434.60852  ]
 [606.3832   ]
 [  3.8676083]
 [  3.647572 ]]
[[436.35538  ]
 [609.80554  ]
 [  3.4085248]
 [  3.5988233]]
[[437.79193  ]
 [612.34326  ]
 [  2.9816465]
 [  3.3691185]]
[[439.13275  ]
 [614.923    ]
 [  2.6264513]
 [  3.198226 ]]
[[441.09348  ]
 [617.5788   ]
 [  2.4823403]
 [  3.0808034]]
[[443.5209   ]
 [620.3194   ]
 [  2.4704483]
 [  3.0071678]]
[[446.27292  ]
 [623.13885  ]
 [  2.5313997]
 [  2.9665334]]
[[449.81934  ]
 [626.6115   ]
 [  2.7511227]
 [  3.07609  ]]
[[453.26434  ]
 [629.3282   ]
 [  2.9013293]
 [  2.9982924]]
[[457.18573  ]
 [632.1336   ]
 [  3.1221433]
 [  2.9565399]]
[[460.78644  ]
 [634.4239   ]
 [  3.2257364]
 [  2.8123202]]
[[464.13766 ]
 [635.8117  ]
 [  3.252906]
 [  2.503948]]
[[468.48508  ]
 [637.25085  ]
 [  3.4898362]
 [  2.273459 ]]
[[472.27756  ]
 [638.7892   ]
 [  3.5553458]
 [  2.1143227]]
[[476.84518  ]
 [639.852    ]
 [  3.7744784]
 [  1.8866988]]
[[480.71066 

...

[[ 5.9671167e+02]
 [ 1.8628738e+01]
 [-2.1650212e+00]
 [-1.4588155e-01]]
[[ 5.9471606e+02]
 [ 1.8701050e+01]
 [-2.1283393e+00]
 [-9.8648869e-02]]
[[ 5.9275458e+02]
 [ 1.8778095e+01]
 [-2.0922165e+00]
 [-6.0615797e-02]]
[[ 5.9021887e+02]
 [ 1.8847895e+01]
 [-2.1882155e+00]
 [-3.2384608e-02]]
[[ 5.8731433e+02]
 [ 1.8904903e+01]
 [-2.3432829e+00]
 [-1.3033448e-02]]
[[ 5.8478632e+02]
 [ 1.8947760e+01]
 [-2.3832729e+00]
 [-9.3505956e-04]]
[[ 5.8194092e+02]
 [ 1.8977526e+01]
 [-2.4833097e+00]
 [ 5.7110707e-03]]
[[ 5.7890460e+02]
 [ 1.8996445e+01]
 [-2.6030154e+00]
 [ 8.5702967e-03]]
[[ 5.7576996e+02]
 [ 1.8419397e+01]
 [-2.7181008e+00]
 [-1.1819950e-01]]
[[ 5.7318707e+02]
 [ 1.8054714e+01]
 [-2.6888344e+00]
 [-1.7155612e-01]]
[[ 5.7038831e+02]
 [ 1.7851004e+01]
 [-2.7126343e+00]
 [-1.7851697e-01]]
[[ 5.6803516e+02]
 [ 1.7760054e+01]
 [-2.6348133e+00]
 [-1.5956134e-01]]
[[ 5.6537970e+02]
 [ 1.7741510e+01]
 [-2.6392860e+00]
 [-1.2903485e-01]]
[[ 5.6251721e+02]
 [ 1.7176685e+01]
 [-2.6875923e+0

 [ 1.5920303e-05]]
[[ 2.3669041e+02]
 [ 2.3000050e+01]
 [-1.9405776e+00]
 [ 5.7287125e-06]]
[[ 2.3493179e+02]
 [ 2.3000027e+01]
 [-1.9011915e+00]
 [-5.8036716e-07]]
[[ 2.3307068e+02]
 [ 2.3000011e+01]
 [-1.8925141e+00]
 [-3.9775641e-06]]
[[ 2.3113663e+02]
 [ 2.3000000e+01]
 [-1.9015059e+00]
 [-5.4335055e-06]]
[[ 2.2915483e+02]
 [ 2.2999994e+01]
 [-1.9188880e+00]
 [-5.4335055e-06]]
[[ 2.2714494e+02]
 [ 2.2999992e+01]
 [-1.9385859e+00]
 [-4.7055346e-06]]
[[ 2.2512117e+02]
 [ 2.2999992e+01]
 [-1.9570259e+00]
 [-3.7349068e-06]]
[[ 2.2368065e+02]
 [ 2.2999994e+01]
 [-1.8452193e+00]
 [-2.7642791e-06]]
[[ 2.2143541e+02]
 [ 2.2999994e+01]
 [-1.9318129e+00]
 [-2.0363084e-06]]
[[ 2.1924770e+02]
 [ 2.2999994e+01]
 [-1.9872066e+00]
 [-1.3083377e-06]]
[[ 2.1770262e+02]
 [ 2.2999996e+01]
 [-1.8914969e+00]
 [-5.8036699e-07]]
[[ 2.1598589e+02]
 [ 2.2999998e+01]
 [-1.8536638e+00]
 [-9.5053167e-08]]
[[ 2.1414052e+02]
 [ 2.2999998e+01]
 [-1.8518682e+00]
 [ 1.4760374e-07]]
[[ 2.1279378e+02]
 [ 2.2999998e+

 [  2.089787  ]]
[[ 75.14156  ]
 [148.91844  ]
 [  0.4524198]
 [  2.142659 ]]
[[ 75.51079  ]
 [150.52133  ]
 [  0.4344098]
 [  2.0258129]]
[[ 75.645004  ]
 [152.24075   ]
 [  0.36942628]
 [  1.9594879 ]]
[[ 76.22306   ]
 [154.05875   ]
 [  0.41458958]
 [  1.9288588 ]]
[[ 77.09426   ]
 [155.95308   ]
 [  0.51343346]
 [  1.9213849 ]]
[[ 78.14     ]
 [157.90204  ]
 [  0.6286635]
 [  1.9273543]]
[[ 78.686386 ]
 [159.88696  ]
 [  0.6108525]
 [  1.9398172]]
[[ 79.48155  ]
 [161.8932   ]
 [  0.6507511]
 [  1.954198 ]]
[[ 80.437     ]
 [163.91017   ]
 [  0.71670914]
 [  1.9677849 ]]
[[ 80.89687  ]
 [165.93076  ]
 [  0.6611135]
 [  1.979213 ]]
[[ 81.61859  ]
 [167.95067  ]
 [  0.6742335]
 [  1.9880224]]
[[ 82.51698  ]
 [169.96768  ]
 [  0.7227571]
 [  1.9942986]]
[[ 83.52361  ]
 [172.56868  ]
 [  0.7842075]
 [  2.1256323]]
[[ 84.000084  ]
 [175.5355    ]
 [  0.71759236]
 [  2.3077278 ]]
[[ 84.71763  ]
 [178.11623  ]
 [  0.7175817]
 [  2.3668215]]
[[ 85.60116  ]
 [181.00246  ]
 [  0.7535056]
 [ 

 [  0.6794031]]
[[149.5122    ]
 [408.01514   ]
 [  0.76633716]
 [  0.5758061 ]]
[[150.56523   ]
 [407.99432   ]
 [  0.8283952 ]
 [  0.44665825]]
[[151.06143  ]
 [408.44434  ]
 [  0.7564853]
 [  0.4473804]]
[[1.5178181e+02]
 [4.0863055e+02]
 [7.4866980e-01]
 [3.9085105e-01]]
[[1.5265871e+02]
 [4.0865082e+02]
 [7.7642781e-01]
 [3.1063062e-01]]
[[1.5304800e+02]
 [4.0916666e+02]
 [6.9262576e-01]
 [3.5505432e-01]]
[[153.71242   ]
 [410.01144   ]
 [  0.68651855]
 [  0.46107402]]
[[154.56795  ]
 [410.4658   ]
 [  0.7231054]
 [  0.4596181]]
[[155.54498   ]
 [411.23938   ]
 [  0.778072  ]
 [  0.52758145]]
[[156.59047   ]
 [411.62628   ]
 [  0.8359602 ]
 [  0.49712703]]
[[157.0794    ]
 [412.34305   ]
 [  0.76083946]
 [  0.54467225]]
[[157.79358   ]
 [412.68613   ]
 [  0.7507372 ]
 [  0.50102895]]
[[158.66562   ]
 [412.7839    ]
 [  0.7769985 ]
 [  0.41373846]]
[[159.63913  ]
 [413.32465  ]
 [  0.8195392]
 [  0.4412305]]
[[160.67075   ]
 [414.1628    ]
 [  0.8654499 ]
 [  0.52715045]]
[[161.729

 [-5.518043e-03]]
[[ 5.3544977e+02]
 [ 5.5101447e+02]
 [ 2.6900306e+00]
 [-1.1675704e-02]]
[[ 5.3787549e+02]
 [ 5.5099432e+02]
 [ 2.6328101e+00]
 [-1.3516014e-02]]
[[ 5.4058142e+02]
 [ 5.5098413e+02]
 [ 2.6486506e+00]
 [-1.2793868e-02]]
[[ 5.4347607e+02]
 [ 5.5098065e+02]
 [ 2.7019031e+00]
 [-1.0774963e-02]]
[[ 5.4589819e+02]
 [ 5.5098126e+02]
 [ 2.6413360e+00]
 [-8.3134510e-03]]
[[ 5.4859937e+02]
 [ 5.5098395e+02]
 [ 2.6542881e+00]
 [-5.9295893e-03]]
[[ 5.5148914e+02]
 [ 5.5098743e+02]
 [ 2.7052577e+00]
 [-3.8873886e-03]]
[[ 5.5449463e+02]
 [ 5.5099097e+02]
 [ 2.7702508e+00]
 [-2.2877944e-03]]
[[ 5.5756189e+02]
 [ 5.5099396e+02]
 [ 2.8345451e+00]
 [-1.1385713e-03]]
[[ 5.6065387e+02]
 [ 5.5099640e+02]
 [ 2.8902824e+00]
 [-3.6983422e-04]]
[[5.6374756e+02]
 [5.5099811e+02]
 [2.9343178e+00]
 [8.8302026e-05]]
[[5.6741791e+02]
 [5.5099927e+02]
 [3.0936561e+00]
 [3.2901767e-04]]
[[5.7085370e+02]
 [5.5100006e+02]
 [3.1677110e+00]
 [4.2219792e-04]]
[[5.7469507e+02]
 [5.5100049e+02]
 [3.3135459